In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Sample data for demonstration
data = {
    'Feature1': np.random.rand(100),
    'Feature2': np.random.rand(100),
    'Target': np.random.randint(0, 2, 100)
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Display first few rows
df.head()

# Data preprocessing
scaler = StandardScaler()
df[['Feature1', 'Feature2']] = scaler.fit_transform(df[['Feature1', 'Feature2']])

# Display processed data
df.head()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Define the generator model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=100),
        layers.Dense(2, activation='tanh')
    ])
    return model

# Define the discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=2),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define the GAN model
def build_gan(generator, discriminator):
    model = tf.keras.Sequential([generator, discriminator])
    return model

# Compile models
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
gan.compile(optimizer='adam', loss='binary_crossentropy')

# GAN training function
def train_gan(epochs=10000, batch_size=64):
    for epoch in range(epochs):
        # Generate fake data
        noise = np.random.randn(batch_size, 100)
        generated_data = generator.predict(noise)
        
        # Combine real and fake data
        real_data = df[['Feature1', 'Feature2']].sample(batch_size).values
        combined_data = np.concatenate([real_data, generated_data], axis=0)
        labels = np.array([1] * batch_size + [0] * batch_size)
        
        # Train discriminator
        d_loss = discriminator.train_on_batch(combined_data, labels)
        
        # Train generator
        noise = np.random.randn(batch_size, 100)
        g_loss = gan.train_on_batch(noise, np.array([1] * batch_size))
        
        if epoch % 1000 == 0:
            print(f'Epoch {epoch}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')

train_gan()


In [ ]:
# Define the conditional generator model
def build_conditional_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=110),
        layers.Dense(2, activation='tanh')
    ])
    return model

# Define the conditional discriminator model
def build_conditional_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_dim=12),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define the Conditional GAN model
def build_conditional_gan(generator, discriminator):
    model = tf.keras.Sequential([generator, discriminator])
    return model

# Compile models
c_generator = build_conditional_generator()
c_discriminator = build_conditional_discriminator()
c_gan = build_conditional_gan(c_generator, c_discriminator)

c_discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
c_gan.compile(optimizer='adam', loss='binary_crossentropy')

# Conditional GAN training function
def train_conditional_gan(epochs=10000, batch_size=64):
    for epoch in range(epochs):
        # Generate fake data with labels
        noise = np.random.randn(batch_size, 100)
        labels = np.random.randint(0, 2, batch_size)
        conditional_input = np.concatenate([noise, labels.reshape(-1, 1)], axis=1)
        generated_data = c_generator.predict(conditional_input)
        
        # Combine real and fake data
        real_data = np.concatenate([df[['Feature1', 'Feature2']], np.random.randint(0, 2, (df.shape[0], 1))], axis=1)
        combined_data = np.concatenate([real_data, np.concatenate([generated_data, labels.reshape(-1, 1)], axis=1)], axis=0)
        real_labels = np.array([1] * batch_size + [0] * batch_size)
        
        # Train discriminator
        c_d_loss = c_discriminator.train_on_batch(combined_data, real_labels)
        
        # Train generator
        noise = np.random.randn(batch_size, 100)
        c_g_loss = c_gan.train_on_batch(np.concatenate([noise, np.random.randint(0, 2, (batch_size, 1))], axis=1), np.array([1] * batch_size))
        
        if epoch % 1000 == 0:
            print(f'Epoch {epoch}, Discriminator Loss: {c_d_loss[0]}, Generator Loss: {c_g_loss}')

train_conditional_gan()
